In [5]:
import numpy as np
import corner
import pandas as pd
import torch
from baobab.sim_utils import add_g1g2_columns
import lenstronomy
print(lenstronomy.__path__)
import os
import h0rton.tdlmc_utils as tdlmc_utils
from h0rton.configs import TrainValConfig, TestConfig
from h0rton.h0_inference import H0Posterior, plot_h0_histogram, h0_utils
from h0rton.trainval_data import XYData
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM

%load_ext autoreload
%autoreload 2
%matplotlib inline

# Plotting params
plt.rcParams.update(plt.rcParamsDefault)
plt.rc('font', family='STIXGeneral', size=20, weight='bold')
plt.rc('xtick', labelsize='medium')
plt.rc('ytick', labelsize='medium')
plt.rc('text', usetex=True)
plt.rc('axes', linewidth=2, titlesize='large', labelsize='large')

['/home/jwp/stage/sl/lenstronomy/lenstronomy']
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
n_test = 200
rung_id = 2

## Full summary, evaluated on Baobab data

In [8]:
label_to_id = {'A1 (0.5 HST orbit)': (4, ''), 
               'A2 (1 HST orbit)': (3, ''), 
               'A3 (2 HST orbits)': (2, ''), 
               'B1 (89 doubles for 1 HST orbit)': (3, '_doubles'), 
               'B2 (89 quads for 1 HST orbit)': (3, '_quads')}
#label_to_id = {'A4 (21,600s)': (6, ''), 'B1, doubles only': (6, '_doubles'), 'B2, quads only': (6, '_quads')}

for label, (version_id, img) in label_to_id.items():
    version_dir = '/home/jwp/stage/sl/h0rton/experiments/v{:d}'.format(version_id)
    summary = pd.read_csv(os.path.join(version_dir, 'summary.csv'), index_col=None)
    true_H0 = 70.0
    true_Om0 = 0.3
    true_cosmo = FlatLambdaCDM(H0=true_H0, Om0=true_Om0)
    outside_rung = summary[ summary['id'] > (200 - 1)].index
    summary.drop(list(outside_rung), inplace=True)
    # Label with null posteriors
    #summary['keep'] = np.logical_and(np.logical_and(~np.isnan(summary['H0_mean']), summary['H0_mean'] > 0), summary['H0_std'] < 6.0)
    summary['keep'] = True #np.logical_and(~np.isnan(summary['H0_mean']), summary['H0_mean'] > 0)
    summary.loc[~summary['keep'], ['H0_mean', 'H0_std']] = -99
    #print(summary[~summary['keep']].shape[0])
    # id is int
    summary['id'] = summary['id'].astype(int)
    
    if img != '':
        summary['is_quad'] = (summary['n_img'] == 4)
        n_test = np.min([len(summary[~summary['is_quad']]), len(summary[summary['is_quad']])])
        if img == '_doubles':
            summary = summary[~summary['is_quad']].iloc[:n_test]
        else:
            summary = summary[summary['is_quad']].iloc[:n_test]
            
    tdlmc_mean = summary['H0_mean'][summary['keep']]
    tdlmc_std = summary['H0_std'][summary['keep']]
    #summary['valid_sample_fraction'] = frac

    summary['g'] = ((summary['H0_mean'] - true_H0)/summary['H0_std'])**2.0
    summary['log_g'] = np.log10(summary['g'])
    summary['p'] = (summary['H0_std']/true_H0)
    summary['a'] = (summary['H0_mean'] - true_H0)/true_H0

    # Test-set-side metrics
    G = tdlmc_utils.get_goodness(tdlmc_mean,tdlmc_std, true_H0)
    P = tdlmc_utils.get_precision(tdlmc_std, true_H0)
    A = tdlmc_utils.get_accuracy(tdlmc_mean, true_H0)
    print(G, np.log10(G), P, A)
    print("Total combined", summary[summary['keep']].shape[0])
    print("Actually discarded", summary[~summary['keep']].shape[0])

    summary['rung_id'] = summary.id.apply(lambda x: 'rung{:d}_seed{:d}'.format(2, x))
    summary[['rung_id', 'H0_mean', 'H0_std']].to_csv('/home/jwp/stage/sl/h0rton/h0rton/tdlmc_utils/rung{:d}_submit_result/H0rton/{:s}.txt'.format(rung_id, label), header=None, index=None, sep=' ', mode='a')

1.2072378463791456 0.08179284192325087 0.09492303696364882 -0.00346699276342569
Total combined 200
Actually discarded 0
1.1367848467251727 0.055678275838218626 0.09654782706946603 -0.0027262909491800958
Total combined 200
Actually discarded 0
1.205895023447009 0.08130950289365438 0.09533901196120378 7.839520842312542e-05
Total combined 200
Actually discarded 0
1.4195045557202568 0.15213679067729954 0.09164239194097494 -0.002167140388336821
Total combined 89
Actually discarded 0
0.8944076069030958 -0.0484645157836915 0.10206174453565184 -0.0031958588270479984
Total combined 89
Actually discarded 0


## Evaluated on TDLMC data

In [7]:
rung_id = 1
label_to_id = {f'BNN Rung{rung_id}': (8, '')}


for label, (version_id, img) in label_to_id.items():
    version_dir = '/home/jwp/stage/sl/h0rton/experiments/v{:d}'.format(version_id)
    summary = pd.read_csv(os.path.join(version_dir, 'summary.csv'), index_col=None)
    true_H0 = summary.iloc[0]['H0']
    true_Om0 = 0.27
    true_cosmo = FlatLambdaCDM(H0=true_H0, Om0=true_Om0)
    #outside_rung = summary[ summary['id'] > 200 ].index
    #summary.drop(list(outside_rung) + [18], inplace=True)
    # Label with null posteriors
    #summary['keep'] = np.logical_and(np.logical_and(~np.isnan(summary['H0_mean']), summary['H0_mean'] > 0), summary['H0_std'] < 6.0)
    #summary['keep'] = np.logical_and(~np.isnan(summary['H0_mean']), summary['H0_mean'] > 0)
    summary['keep'] = True
    summary.loc[~summary['keep'], ['H0_mean', 'H0_std']] = -99
    # id is int
    summary['id'] = summary['id'].astype(int)
            
    print("Number of lenses: ", summary.shape[0])
    tdlmc_mean = summary['H0_mean'][summary['keep']]
    tdlmc_std = summary['H0_std'][summary['keep']]
    #summary['valid_sample_fraction'] = frac

    summary['g'] = ((summary['H0_mean'] - true_H0)/summary['H0_std'])**2.0
    summary['log_g'] = np.log10(summary['g'])
    summary['p'] = (summary['H0_std']/true_H0)
    summary['a'] = (summary['H0_mean'] - true_H0)/true_H0

    # Test-set-side metrics
    G = tdlmc_utils.get_goodness(tdlmc_mean,tdlmc_std, true_H0)
    P = tdlmc_utils.get_precision(tdlmc_std, true_H0)
    A = tdlmc_utils.get_accuracy(tdlmc_mean, true_H0)
    print(G, np.log10(G), P, A)
    print("Actually discarded", summary[summary['H0_mean']<0].shape[0])

    summary['rung_id'] = summary.id.apply(lambda x: 'rung{:d}_seed{:d}'.format(rung_id, x))
    summary[['rung_id', 'H0_mean', 'H0_std']].to_csv('/home/jwp/stage/sl/h0rton/h0rton/tdlmc_utils/rung{:d}_submit_result/H0rton/{:s}.txt'.format(rung_id, label), header=None, index=None, sep=' ', mode='a')

Number of lenses:  16
3997.056259071666 3.6017402610528073 0.06959427650369562 -0.007154437429482564
Actually discarded 0


In [11]:
rung_id = 2
label_to_id = {f'BNN Rung{rung_id}': (8, '')}


for label, (version_id, img) in label_to_id.items():
    version_dir = '/home/jwp/stage/sl/h0rton/experiments/v{:d}'.format(version_id)
    summary = pd.read_csv(os.path.join(version_dir, 'summary.csv'), index_col=None)
    true_H0 = summary.iloc[0]['H0']
    true_Om0 = 0.27
    true_cosmo = FlatLambdaCDM(H0=true_H0, Om0=true_Om0)
    #outside_rung = summary[ summary['id'] > 200 ].index
    #summary.drop(list(outside_rung) + [18], inplace=True)
    # Label with null posteriors
    #summary['keep'] = np.logical_and(np.logical_and(~np.isnan(summary['H0_mean']), summary['H0_mean'] > 0), summary['H0_std'] < 6.0)
    #summary['keep'] = np.logical_and(~np.isnan(summary['H0_mean']), summary['H0_mean'] > 0)
    summary['keep'] = True
    summary.loc[~summary['keep'], ['H0_mean', 'H0_std']] = -99
    # id is int
    summary['id'] = summary['id'].astype(int)
            
    print("Number of lenses: ", summary.shape[0])
    tdlmc_mean = summary['H0_mean'][summary['keep']]
    tdlmc_std = summary['H0_std'][summary['keep']]
    #summary['valid_sample_fraction'] = frac
    summary['g'] = ((summary['H0_mean'] - true_H0)/summary['H0_std'])**2.0
    summary['log_g'] = np.log10(summary['g'])
    summary['p'] = (summary['H0_std']/true_H0)
    summary['a'] = (summary['H0_mean'] - true_H0)/true_H0

    # Test-set-side metrics
    G = tdlmc_utils.get_goodness(tdlmc_mean,tdlmc_std, true_H0)
    P = tdlmc_utils.get_precision(tdlmc_std, true_H0)
    A = tdlmc_utils.get_accuracy(tdlmc_mean, true_H0)
    print(G, np.log10(G), P, A)
    print("Actually discarded", summary[summary['H0_mean']<0].shape[0])

    summary['rung_id'] = summary.id.apply(lambda x: 'rung{:d}_seed{:d}'.format(rung_id, x))
    summary[['rung_id', 'H0_mean', 'H0_std']].to_csv('/home/jwp/stage/sl/h0rton/h0rton/tdlmc_utils/rung{:d}_submit_result/H0rton/{:s}.txt'.format(rung_id, label), header=None, index=None, sep=' ', mode='a')

Number of lenses:  16
163.48730759554948 2.213484041674091 0.05745207506179319 0.08651182715086295
Actually discarded 0


In [13]:
summary[['H0_mean', 'H0_std']]

,H0_mean,H0_std
0,61.395633,3.205568
1,84.845738,3.301239
2,82.810214,2.392886
3,88.952397,0.926670
4,75.972218,3.802021
5,64.966145,4.891438
6,57.702937,1.686952
7,50.584973,0.524114
8,78.274419,8.481272
9,NaN,NaN
